## Setting Up Super Mario

In [ ]:
pip install gym_super_mario_bros==7.3.0 nes_py

In [14]:
#import game
import gym_super_mario_bros
#Import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
#Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [15]:
#Setting Up Game
env = gym_super_mario_bros.make('SuperMarioBros-v3')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [3]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

In [ ]:
#Restart Game or Not
done = True
#Looping through each frame in the Game
for step in range(100000):
    if done:
        #Start the Game
        env.reset()
        #Do random action
    state , reward , done , info = env.step(env.action_space.sample()) 
    # Show Game on the screen
    env.render()
#Close the game                                        
env.close()

In [ ]:
env.close()

## Preprocess Environment

In [ ]:
conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

In [ ]:
 $ conda update -n base -c defaults conda

In [ ]:
conda install -c anaconda pip

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
pip install matplotlib

In [5]:
#Import GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
#Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack , DummyVecEnv
#Import Matplotlib to show the impact of frame stacking
import matplotlib.pyplot as plt

In [6]:
# 1 . Create base environment
env = gym_super_mario_bros.make('SuperMarioBros-v3')
#2 . Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
#3 . GrayScale
env = GrayScaleObservation(env , keep_dim = True)
#4 . Wrap Inside Dummy Environment
env = DummyVecEnv([lambda: env])
#5. Stack the frames
env = VecFrameStack(env, 4, channels_order = 'last')

In [7]:
state = env.reset()

In [15]:
state.shape

(1, 240, 256, 4)

In [11]:
state , reward , done , info = env.step([env.action_space.sample()]) 

In [ ]:
#Use plt to show the game frame
plt.imshow(state)

In [ ]:
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx + 1)
    plt.imshow(state[0][:,:,idx])
plt.show()   

## Train the RL Model

In [ ]:
pip install stable-baselines3

In [6]:
import stable_baselines3
import gym

In [7]:
#Import os for file path management
import os
#Import PPO for algos
from stable_baselines3 import PPO
#Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [8]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [9]:
CHECKPOINT_DIR = 'D:/train/'
LOG_DIR = 'D:/logs/'

In [10]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [16]:
#This is the AI Model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps= 512,
            )

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
%%time
model.learn(total_timesteps= 1000000, callback=callback)

Logging to D:/logs/PPO_2


D:\anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 59  |
|    iterations      | 1   |
|    time_elapsed    | 8   |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 11           |
|    iterations           | 2            |
|    time_elapsed         | 92           |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 6.456487e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.95        |
|    explained_variance   | -0.00257     |
|    learning_rate        | 1e-06        |
|    loss                 | 88.2         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000404    |
|    value_loss           | 295          |
------------------------------------------
-----------------------

## Test it out

In [14]:
#Load Model
model  = PPO.load('D:/train/best_model_500000')

In [ ]:
#Start the game
state = env.reset()
#Loop through the game
while True:
    
    action, _ = model.predict(state)
    state , reward , done , info = env.step(action)
    env.render()

In [16]:
env.close()